In [1]:
# !wget https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py

--2024-06-27 17:02:52--  https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3832 (3.7K) [text/plain]
Saving to: ‘minsearch.py’

minsearch.py        100%[===================>]   3.74K  --.-KB/s    in 0.002s  

2024-06-27 17:02:53 (1.51 MB/s) - ‘minsearch.py’ saved [3832/3832]



In [1]:
import minsearch
import json

In [2]:
with open('01-intro/documents.json', 'rt') as f_in:
    docs_raw = json.load(f_in)

In [3]:
documents = []

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

In [4]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [5]:
# indexing 
index = minsearch.Index(
    text_fields=['question', "text", "section"],
    keyword_fields=["course"]
)

In [54]:
q = "The Course has already strated, can I still join?"

In [55]:
index.fit(documents)

In [56]:
boost = {'question': 3.0, 'section':0.5}

results = index.search(
    query=q,
    filter_dict={'course': "data-engineering-zoomcamp"},
    boost_dict=boost,
    num_results=5
)

In [57]:
results

[{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
  'section': 'General course-related questions',
  'question': 'Course - When will the course start?',
  'course': 'data-engineerin

## Getting data from open source LLMS

In [20]:
import os
from openai import OpenAI
from configs import api_key 


In [21]:
client = OpenAI(api_key=api_key.API_KEY)

In [58]:
q = "The Course has already strated, can I still join?"

In [59]:
response = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {"role": "user", "content": q}
    ])

In [60]:
response.choices[0].message.content

"Yes, it's often possible to join a course after it has started, but the specifics depend on the policies of the institution or organization offering the course. Here are a few steps you can take:\n\n1. **Contact the Instructor or Coordinator**: Reach out to the course instructor or the coordinator to express your interest in joining. They can provide information on whether late enrollment is allowed and any steps you need to take.\n\n2. **Check the Course Syllabus and Requirements**: Make sure you're aware of any missed content and requirements. You may need to catch up on lectures, assignments, or readings.\n\n3. **Access Course Materials**: Find out if the materials (recorded lectures, readings, assignments) are available online so you can review what you've missed.\n\n4. **Understand the Policy**: Some courses may have strict policies on attendance or participation that could impact your ability to join late. Understand these policies to avoid any issues.\n\n5. **Catch Up Quickly**

In [61]:
prompt_template = """
    Youre a course teaching assistant. Answer the question based on the context from the FAQ database.
    use only the facts from the context when answering the question. if the context doesnt contain the answer, output NONE

    QUESTION: {question}

    CONTEXT:
    {context}
"""

In [62]:
context = ""

for doc in results:
    context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"

In [31]:
print(context)

section: General course-related questions
question: Course - Can I still join the course after the start date?
answer: Yes, even if you don't register, you're still eligible to submit the homeworks.
Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.

section: General course-related questions
question: Course - When will the course start?
answer: The purpose of this document is to capture frequently asked technical questions
The exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1
Subscribe to course public Google Calendar (it works from Desktop only).
Register before the course starts using this link.
Join the course Telegram channel with announcements.
Don’t forget to register in DataTalks.Club's Slack and join the channel.

section: General course-related questions
question: Course - Can I follow the course after it finishes?
answer: 

In [63]:
prompt = prompt_template.format(question=q, context=context).strip()
# prompt

In [64]:
print(prompt)

Youre a course teaching assistant. Answer the question based on the context from the FAQ database.
 use only the facts from the context when answering the question. if the context doesnt contain the answer, output NONE

QUESTION: The Course has already strated, can I still join?

CONTEXT:
section: General course-related questions
question: Course - Can I still join the course after the start date?
answer: Yes, even if you don't register, you're still eligible to submit the homeworks.
Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.

section: General course-related questions
question: Course - When will the course start?
answer: The purpose of this document is to capture frequently asked technical questions
The exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1
Subscribe to course public Google Calendar (it works from Desktop only).


In [65]:
response = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {"role": "user", "content": prompt}
    ])

In [66]:
print(response.choices[0].message.content)

Yes, even if the course has already started, you can still join and submit the homework. However, be mindful of the deadlines for submitting final projects.


### Putting everything together

In [81]:
def search(query):
    boost = {'question': 3.0, 'section':0.5}

    res = index.search(
        query=query,
        filter_dict={'course': "data-engineering-zoomcamp"},
        boost_dict=boost,
        num_results=10
    )

    return res


search("How do i run Kafka?")

[{'text': 'In the project directory, run:\njava -cp build/libs/<jar_name>-1.0-SNAPSHOT.jar:out src/main/java/org/example/JsonProducer.java',
  'section': 'Module 6: streaming with kafka',
  'question': 'Java Kafka: How to run producer/consumer/kstreams/etc in terminal',
  'course': 'data-engineering-zoomcamp'},
 {'text': "Solution from Alexey: create a virtual environment and run requirements.txt and the python files in that environment.\nTo create a virtual env and install packages (run only once)\npython -m venv env\nsource env/bin/activate\npip install -r ../requirements.txt\nTo activate it (you'll need to run it every time you need the virtual env):\nsource env/bin/activate\nTo deactivate it:\ndeactivate\nThis works on MacOS, Linux and Windows - but for Windows the path is slightly different (it's env/Scripts/activate)\nAlso the virtual environment should be created only to run the python file. Docker images should first all be up and running.",
  'section': 'Module 6: streaming wi

In [82]:
def build_prompt(query, search_results):
    prompt_template = """
        You're a course teaching assistant. Answer the question based on the context from the FAQ database.
        use only the facts from the context when answering the question. 

        QUESTION: {question}

        CONTEXT:
        {context}
    """.strip()

    context = ""

    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
        
    prompt = prompt_template.format(question=query, context=context).strip()

    return  prompt


In [83]:
def llm(prompt):
    
    response = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {"role": "user", "content": prompt}
    ])

    return response.choices[0].message.content

In [86]:

query = "How do i run kafka?"

def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)

    answer = llm(prompt)

    return answer


In [89]:
rag("who are the instructors of this course??")

'The provided context does not specify the names of the instructors for this course. You may want to check the course homepage or other official resources for this information.'

In [90]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [91]:
from elasticsearch import Elasticsearch

In [92]:
from elasticsearch import Elasticsearch
es_client = Elasticsearch('http://localhost:9200')

In [93]:
es_client.info()

ObjectApiResponse({'name': 'f9b0787a0cca', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'KIZ49sIQT-qL3ghxYY_gAg', 'version': {'number': '8.4.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73', 'build_date': '2022-10-04T07:17:24.662462378Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [95]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-questions"

es_client.indices.create(
    index=index_name,
    body=index_settings
)


ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [97]:
from tqdm.auto import tqdm

In [99]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████| 948/948 [00:08<00:00, 107.40it/s]


In [105]:
query = 'i just discovered this course, can i still join?'

In [ ]:
def elastic_search(query):
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^3", "text", "section"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "data-engineering-zoomcamp"
                    }
                }
            }
        }
    }

    response = es_client.search(index=index_name, body=search_query)

    results_doc = []

    for hit in response['hits']['hits']:
        results_doc.append(hit['_source'])

    return results_doc



In [110]:
elastic_search('How do i run Kafka?')

[{'text': "Answer: To run the provided code, ensure that the 'dlt[duckdb]' package is installed. You can do this by executing the provided installation command: !pip install dlt[duckdb]. If you’re doing it locally, be sure to also have duckdb pip installed (even before the duckdb package is loaded).",
  'section': 'Workshop 1 - dlthub',
  'question': 'How do I install the necessary dependencies to run the code?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "Answer: To run the provided code, ensure that the 'dlt[duckdb]' package is installed. You can do this by executing the provided installation command: !pip install dlt[duckdb]. If you’re doing it locally, be sure to also have duckdb pip installed (even before the duckdb package is loaded).",
  'section': 'Workshop 1 - dlthub',
  'question': 'How do I install the necessary dependencies to run the code?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "Answer: To run the provided code, ensure that the 'dlt[duckdb]' package

In [111]:

query = "How do i run kafka?"

def rag(query):
    # search_results = search(query)
    search_results = elastic_search(query)
    
    prompt = build_prompt(query, search_results)

    answer = llm(prompt)

    return answer


In [114]:
rag('How do i run kafka?')

'To run Kafka, you can execute the Java Kafka producer, consumer, or other components from the terminal. Here are the steps you can follow:\n\n1. Navigate to your project directory.\n2. Run the following command in the terminal:\n   ```sh\n   java -cp build/libs/<jar_name>-1.0-SNAPSHOT.jar:out src/main/java/org/example/JsonProducer.java\n   ```\n\nMake sure to replace `<jar_name>` with the actual name of your JAR file. This command will execute the specified Java file (`JsonProducer.java` in this case) which should contain the logic for your Kafka producer, consumer, or other components.'